In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets peft wandb sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [4]:
# Check GPU availability
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Please enable it in runtime settings.")

GPU is available!
GPU name: NVIDIA A100-SXM4-40GB


In [5]:
import torch
torch.cuda.empty_cache()

# Load Dataset

In [6]:
# File paths
training_set = "/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-train.csv"
validation_set = "/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-val.csv"
testing_set = "/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-test.csv"

# Load datasets
train_data = pd.read_csv(training_set)
val_data = pd.read_csv(validation_set)
test_data = pd.read_csv(testing_set)

# Add prompt column to the datasets
for dataset in [train_data, val_data, test_data]:
    dataset['prompt'] = dataset.apply(
        lambda row: f"Symptoms: {row['symptoms']}\nPredict the disease:",
        axis=1
    )

# Create mappings for text-based labels
unique_labels = train_data['disease_label'].unique()
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for idx, label in enumerate(unique_labels)}

# Map text labels to numeric indices
for dataset in [train_data, val_data, test_data]:
    dataset['target'] = dataset['disease_label'].map(label_to_id)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

disease_counts = train_data['disease_label'].value_counts()
print("Disease Distribution in Training Data:")
print(disease_counts)


Number of training samples: 44164
Number of validation samples: 8810
Number of testing samples: 8846
Disease Distribution in Training Data:
disease_label
hypertensive disease                  1098
biliary calculus                      1087
hyperlipidemia                        1068
colitis                               1003
paroxysmal dyspnea                     990
coronary arteriosclerosis              941
osteomyelitis                          923
infection                              913
failure heart                          896
gastritis                              875
overload fluid                         862
malignant neoplasms                    857
hepatitis                              854
delirium                               852
deep vein thrombosis                   847
manic disorder                         840
dependence                             838
stenosis aortic valve                  838
effusion pericardial                   837
tachycardia sinus            

# Load LLM

In [7]:
# Import the required classes
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load ClinicalBERT model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # ClinicalBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels)
)

print("ClinicalBERT model and tokenizer loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ClinicalBERT model and tokenizer loaded successfully!


# LoRA (Low-Rank Adaptation)

In [8]:

# Configure LoRA for BioBERT
lora_config = LoraConfig(
    r=8,  # Low-rank adaptation rank
    lora_alpha=32,  # Scaling factor
    target_modules=["query", "value"],  # LoRA applied to attention layers
    lora_dropout=0.1,  # Dropout rate for regularization
    task_type="SEQ_CLS"  # Sequence classification task
)

model = get_peft_model(model, lora_config)
print("LoRA applied successfully to ClinicalBERT!")


LoRA applied successfully to ClinicalBERT!


# Tokenization function

In [9]:
# Tokenization function for the dataset
def tokenize_function(examples):
    # Tokenize the 'prompt' column
    tokenized_inputs = tokenizer(
        examples['prompt'],  # Input text (symptoms prompt)
        truncation=True,
        padding='max_length',
        max_length=256,  # Adjust as needed
        return_tensors="pt"
    )
    # Add 'labels' field (numeric targets)
    tokenized_inputs["labels"] = examples["target"]
    return tokenized_inputs

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the dataset format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/44164 [00:00<?, ? examples/s]

Map:   0%|          | 0/8810 [00:00<?, ? examples/s]

Map:   0%|          | 0/8846 [00:00<?, ? examples/s]

# Training Arguments

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results_clinicalbert',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    learning_rate=5e-5,
    num_train_epochs= 7,
    weight_decay=0.001,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir='./logs_clinicalbert',
    logging_steps=10,
    fp16=True,  # Mixed precision
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-11-bc59b6678d02>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.732800,1.671751
2,1.023700,0.845118
3,0.665000,0.492177
4,0.569100,0.369711
5,0.515700,0.299773
6,0.497800,0.276962
7,0.304600,0.271152


TrainOutput(global_step=19327, training_loss=0.9988335127442414, metrics={'train_runtime': 1070.5759, 'train_samples_per_second': 288.768, 'train_steps_per_second': 18.053, 'total_flos': 4.087155522022195e+16, 'train_loss': 0.9988335127442414, 'epoch': 7.0})

# Training and Validation Loss Graph

In [14]:
"""
import matplotlib.pyplot as plt

# Data from the table
epochs = [1, 2]
training_loss = [2.1982, 2.4477, 1.8128, 1.3208, 1.1841, 1.1038]
validation_loss = [1.836972, 2.2917, 1.5439, 1.1537, 0.9580, 0.9067]

# Plot the graph
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, label="Training Loss", marker="o")
plt.plot(epochs, validation_loss, label="Validation Loss", marker="o")

# Add labels, title, and legend
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()
"""

'\nimport matplotlib.pyplot as plt\n\n# Data from the table\nepochs = [1, 2]\ntraining_loss = [2.1982, 2.4477, 1.8128, 1.3208, 1.1841, 1.1038]\nvalidation_loss = [1.836972, 2.2917, 1.5439, 1.1537, 0.9580, 0.9067]\n\n# Plot the graph\nplt.figure(figsize=(10, 6))\nplt.plot(epochs, training_loss, label="Training Loss", marker="o")\nplt.plot(epochs, validation_loss, label="Validation Loss", marker="o")\n\n# Add labels, title, and legend\nplt.xlabel("Epoch")\nplt.ylabel("Loss")\nplt.title("Training and Validation Loss Over Epochs")\nplt.legend()\nplt.grid(True)\n\n# Show the plot\nplt.show()\n'

In [15]:
"""
model.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biobert')
tokenizer.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biobert')
print("Fine-tuned BioBERT model and tokenizer saved successfully!")
"""

'\nmodel.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biobert\')\ntokenizer.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biobert\')\nprint("Fine-tuned BioBERT model and tokenizer saved successfully!")\n'

# Evaluate the fine-tuned model directly

In [16]:
from datasets import Dataset

# Load the test dataset
test_file_path = '/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-test.csv'
test_data = pd.read_csv(test_file_path)

# Add prompt column (if not already present)
test_data['prompt'] = test_data.apply(
    lambda row: f"Symptoms: {row['symptoms']}\nPredict the disease:",
    axis=1
)

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_data)

In [17]:
import re

def predict_disease(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256).to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=-1).item()
        return id_to_label[predicted_class_id]  # Map numeric index back to text label

# Test Predictions
test_data['predicted_disease'] = test_data['prompt'].apply(predict_disease)

# Print Predictions
for index, row in test_data.iterrows():
    print(f"Prompt: {row['prompt']}")
    print(f"Predicted Disease: {row['predicted_disease']}")


Streaming output truncated to the last 5000 lines.
Predict the disease:
Predicted Disease: failure heart
Prompt: Symptoms: Patient shows symptoms as follows: have orthopnea; have dyspnea on exertion.
Predict the disease:
Predicted Disease: failure heart
Prompt: Symptoms: Patient shows symptoms as follows: have patient non compliance; have distended abdomen; have cough.
Predict the disease:
Predicted Disease: hepatitis C
Prompt: Symptoms: Patient shows symptoms as follows: have lesion; have hematochezia; have fremitus; have pain.
Predict the disease:
Predicted Disease: adenocarcinoma
Prompt: Symptoms: Patient shows symptoms as follows: have hypothermia, natural; have unresponsiveness; have asthenia; have haemorrhage; have hyperkalemia; have oliguria.
Predict the disease:
Predicted Disease: kidney failure acute
Prompt: Symptoms: Patient shows symptoms as follows: have lameness; have incoherent; have clammy skin; have claudication; have hypotension.
Predict the disease:
Predicted Disease:

# Save fine-tuned Model

In [18]:
"""
# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt')
tokenizer.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt')
print("Fine-tuned BioGPT model and tokenizer saved successfully!")
"""

'\n# Save the fine-tuned model\nmodel.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt\')\ntokenizer.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt\')\nprint("Fine-tuned BioGPT model and tokenizer saved successfully!")\n'

# Generate Classification Report

In [19]:
from sklearn.metrics import classification_report
import pandas as pd

# Assume `test_data` is your test dataset after generating predictions
# The test_data dataframe should already have columns: 'Label' (true labels) and 'Predicted' (model predictions)

# True labels and predicted labels
true_labels = test_data['disease_label']
predicted_labels = test_data['predicted_disease']

# Generate classification report
report = classification_report(true_labels, predicted_labels, output_dict=False)
print(report)

# Save classification report to a file
report_dict = classification_report(true_labels, predicted_labels, output_dict=True)
pd.DataFrame(report_dict).transpose().to_csv('/content/drive/MyDrive/P2/T1/Dataset/combined/LLM/BioClinicalBERT-c-class.csv', index=True)
print("Classification report saved to '/content/drive/MyDrive/P2/T1/Dataset/combined/LLM/BioClinicalBERT-c-class.csv")


                                    precision    recall  f1-score   support

                    adenocarcinoma       0.98      0.95      0.96       171
                          adhesion       0.93      0.93      0.93       168
                     affect labile       0.99      0.91      0.95        79
                         arthritis       0.99      0.92      0.96       131
                            asthma       0.89      0.95      0.92        83
      benign prostatic hypertrophy       1.00      0.89      0.94        79
                  biliary calculus       0.86      0.83      0.85       194
                  carcinoma breast       0.95      0.96      0.96       170
chronic obstructive airway disease       0.85      0.91      0.88       139
                         cirrhosis       0.95      0.93      0.94       178
                           colitis       0.87      0.89      0.88       178
         coronary arteriosclerosis       0.75      0.81      0.78       162
           

In [20]:
"""
output_path = '/content/drive/MyDrive/P2/LLM/ClinicalBERT_Predictions.csv'
test_data.to_csv(output_path, index=False)
print(f"Predictions saved to: {output_path}")
"""

'\noutput_path = \'/content/drive/MyDrive/P2/LLM/ClinicalBERT_Predictions.csv\'\ntest_data.to_csv(output_path, index=False)\nprint(f"Predictions saved to: {output_path}")\n'